In [1]:
import uproot
import awkward as ak
import numpy as np
import hist
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

In [2]:
ar_raw = uproot.concatenate([f"root://eos.grif.fr//eos/grif/cms/llr/store/user/tcuisset/boostedTaus/2018_MC/GluGluToXToZZTo2B2Tau_M-3000/NANO_NANO_{i}.root:Events" for i in [1]],
    filter_name=["boostedTau*", "Muon_*", "Electron_*", "GenPart_pdgId", "GenPart_genPartIdxMother", "GenPart_eta", "GenPart_phi", "GenPart_pt", "nGenVisTau", "GenVisTau*"],
    how="zip")

In [3]:
def genPairType_branch(ar):
    """ select etau, mutau, tautau at genlevel"""
    tautau = ar.nGenVisTau == 2

    mutau_genMuon_bool = (abs(ar.GenPart.pdgId) == 13) & (abs(ar.GenPart[ar.GenPart.genPartIdxMother].pdgId) == 15)
    mutau_genMuon_idx = ak.firsts(ak.local_index(ar.GenPart)[mutau_genMuon_bool]) # index of gen muon in GenPart collection
    mutau = (ar.nGenVisTau == 1) & ak.any(mutau_genMuon_bool, axis=-1)
    
    etau_genMuon_bool = (abs(ar.GenPart.pdgId) == 11) & (abs(ar.GenPart[ar.GenPart.genPartIdxMother].pdgId) == 15)
    etau_genMuon_idx = ak.firsts(ak.local_index(ar.GenPart)[etau_genMuon_bool]) # index of gen muon in GenPart collection
    etau = (ar.nGenVisTau == 1) & ak.any(etau_genMuon_bool, axis=-1)
    
    genPairType =  ak.where(mutau, 0, ak.where(etau, 1, ak.where(tautau, 2, -1)))
    genLeptonIdx = ak.where(mutau, mutau_genMuon_idx, etau_genMuon_idx) # for tautau will just put None
    return ak.with_field(ak.with_field(ar, genPairType, where="genPairType"), genLeptonIdx, "genLeptonIdx")

# def genDaus(ar):
#     genpart = ak.firsts(ar.GenPart[ak.drop_none(ak.singletons(ar.genLeptonIdx))])
#     ar = ak.with_field(
#         ar,
#         ak.where((ar.genPairType >= 0) & (ar.genPairType <= 1), 
#                      genpart[["pt", "eta", "phi"]],
#                      ak.where(ar.genPairType == 2,
#                               ak.firsts(ar.GenVisTau)[["pt", "eta", "phi"]],
#                               None)
#                      ),
#         "genDau1"
#     )
#     return ar

# def genFilterDetectorAcceptance(ar):
    

def eff(branch_num, filter_denom):
    try:
        return ak.count_nonzero(branch_num) / ak.count_nonzero(filter_denom)
    except:
        return ak.count_nonzero(branch_num) / len(filter_denom)

def delta_phi(a, b):
    """Compute difference in angle given two angles a and b

    Returns a value within [-pi, pi)
    """
    return (a - b + np.pi) % (2 * np.pi) - np.pi

def delta_r(eta1, phi1, eta2, phi2):
    r"""Distance in (eta,phi) plane given two pairs of (eta,phi)

    :math:`\sqrt{\Delta\eta^2 + \Delta\phi^2}`
    """
    deta = eta1 - eta2
    dphi = delta_phi(phi1, phi2)
    return np.hypot(deta, dphi)

class DotDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
deeptau=DotDict(
    vsjet=DotDict(VVVLoose=1, VVLoose=2, VLoose=3, Loose=4, Medium=5,
        Tight=6, VTight=7, VVTight=8),
    vse=DotDict(VVVLoose=1, VVLoose=2, VLoose=3, Loose=4, Medium=5,
        Tight=6, VTight=7, VVTight=8),
    vsmu=DotDict(VLoose=1, Loose=1, Medium=3, Tight=4),
)


def match_muon_gen(ar):
    """ Match muon to gen """
    gen_muons = ak.firsts(ar.GenPart[ak.singletons(ar.genLeptonIdx)])
    matched = (delta_r(gen_muons.eta, gen_muons.phi, ar.Muon.eta, ar.Muon.phi) < 0.1)
    return ak.firsts(ar.Muon[matched])

def match_boostedtau_gen(ar):
    """ Match boostedtau to gen """
    gen_taus = ak.firsts(ar.GenVisTau)
    matched = (delta_r(gen_taus.eta, gen_taus.phi, ar.boostedTau.eta, ar.boostedTau.phi) < 0.4)
    return ak.firsts(ar.boostedTau[matched])


In [37]:
ar_tautau.GenVisTau[:, 0]

<Array [{status: 15, charge: -1, ...}, ...] type='6696 * {status: uint8, ch...'>

In [4]:
ar = genPairType_branch(ar_raw)
ar

<Array [{nGenVisTau: 2, ...}, ..., {...}] type='16000 * {nGenVisTau: int32,...'>

In [5]:
ar_tautau = ar[ar.genPairType == 2]
ar_mutau = ar[ar.genPairType == 0]

In [6]:
ar_mutau.Electron.vidNestedWPBitmap

<Array [[605030436], [], [], ..., [...], [611436644]] type='3556 * var * int32'>

In [28]:
print(f"{ar_mutau.Electron.vidNestedWPBitmap[0, 0]:0{10*3}b}")
print(f"{0x37F:0{10*3}b}")

100100000100000000100000100100
000000000000000000001101111111


In [34]:
cut = 0
for bitn in [0,1,2,3,4,5,6,8,9]:
    cut |= (2 << (bitn*3)) # 2 is for loose, 3 is for 3 bits per cut
f"{cut:0{10*3}b}"

'010010000010010010010010010010'

In [25]:
f"{4:b}"

'100'

In [7]:
(ar_mutau.Electron.vidNestedWPBitmap[0, 0]>>(7*3))&0x7 # fail=0, veto=1, loose, medium, tight=4

0

In [11]:
ar_mutau.Electron.cutBased

<Array [[0], [], [], [], ..., [0, 0], [0, 0], [1]] type='3556 * var * uint8'>

In [20]:
ar_tautau.genLeptonIdx

<Array [29, None, None, None, ..., None, None, None, 94] type='6696 * ?int64'>

In [30]:
ak.firsts(ar_mutau.GenPart[ak.drop_none(ak.singletons(ar_mutau.genLeptonIdx))])

<Array [{genPartIdxMother: 15, ...}, ...] type='3556 * ?{genPartIdxMother: ...'>

In [21]:

evt = ar_tautau[1]

In [22]:
evt.boostedTau.genPartFlav

<Array [5, 5, 0] type='3 * uint8'>

In [23]:
evt.boostedTau.genPartIdx

<Array [0, 1, -1] type='3 * int16'>

In [27]:
evt.GenVisTau[0]

<Record {status: 2, charge: -1, ...} type='{status: uint8, charge: int16, g...'>

In [64]:
b = uproot.open("/grid_mnt/data__data.polcms/cms/cuisset/cmt/PreprocessRDF/ul_2018_ZZ_v12/GluGluToXToZZTo2B2Tau_M3000_BT/cat_base/prod_240923_BT_test/data_4.root:Events").arrays(filter_name=["boostedTau*", "Muon_*", "Electron_*", "GenPart_pdgId", "GenPart_genPartIdxMother", "GenPart_eta", "GenPart_phi", "GenPart_pt", "nGenVisTau", "GenVisTau*",
        "GenPairType", "genDau*", "cutflow*", "GenVisTau*"],
    how="zip")
b

<Array [{nGenVisTau: 2, ...}, ..., {...}] type='3935 * {nGenVisTau: int32, ...'>

In [65]:
b.GenPairType

<Array [2, 3, 2, -1, 2, 2, -1, ..., 1, 0, 2, -1, 3, 2, 1] type='3935 * int32'>

In [66]:
b[(b.GenPairType >= 0) & (b.GenPairType <= 2)].genDau1_genPartIdx

<Array [0, -1, 0, 0, 50, 34, 0, ..., 0, 43, 44, 0, 0, 42] type='2973 * int32'>

In [75]:
b[b.GenPairType == 2].boostedTau.genPartFlav

<Array [[5, 0, 0], [5, 0, 5], ..., [...], [5, 0, 0]] type='1648 * var * uint8'>

In [73]:
b[b.GenPairType == 2].boostedTau.genPartIdx

<Array [[1, 57, -1], [0, ..., 2], ..., [0, -1, -1]] type='1648 * var * int16'>

In [70]:
b[b.GenPairType == 2].GenVisTau[0][1]

<Record {status: 11, charge: -1, ...} type='{status: uint8, charge: int16, ...'>

In [68]:
b[b.GenPairType == 2].genDau2_genPartIdx

<Array [1, -1, 1, 1, 1, 1, 1, ..., -1, 1, -1, 1, 1, 1, 1] type='1648 * int32'>